In [ ]:
# $ conda install -c conda-forge ipyvolume
import ewald_summation as es
import matplotlib.pyplot as plt
import numpy as np
import ipyvolume as ipv

In [ ]:
def initializer(n_particles, n_dim):
    masses = np.array([39.948] * n_particles)
    q_0 = np.arange(n_particles * n_dim).reshape(n_particles, n_dim)
    v_0 = np.zeros((n_particles, n_dim))
    particle_types = [0] * n_particles
    lj_mixing_conditions = tuple([(3.405, 3.405**6, 0.238)])
    n_particles_tpyes = 1
    return q_0, v_0 * masses[:, None], masses, particle_types, lj_mixing_conditions, n_particles_tpyes

def random_initializer(n_particles, n_dim):
    masses = np.array([39.948] * n_particles)
    q_0 = np.random.random((n_particles, n_dim)) * 20
    v_0 = np.zeros((n_particles, n_dim))
    particle_types = [0] * n_particles
    lj_mixing_conditions = tuple([(3.405, 3.405**6, 0.238)])
    n_particles_tpyes = 1
    return q_0, v_0 * masses[:, None], masses, particle_types, lj_mixing_conditions, n_particles_tpyes

cutoff = 3.405 * 3.5
damping = 0.01
n_steps = 15000
n_particles = 1000
n_dim = 3

In [ ]:
q_cuda, p_cuda = es.potentials.lagevin_harmonic_lj_cuda(*initializer(n_particles, n_dim),
                                                        n_steps, cutoff, time_step=0.001, damping=damping)

In [ ]:
ipv.figure()
s = ipv.scatter(q_cuda[:, :, 0], q_cuda[:, :, 1], q_cuda[:, :, 2], marker='sphere', size=10)
ipv.xyzlim(-10, 10)
ipv.animation_control(s) # shows controls for animation controls
ipv.show()

In [ ]:
n_particles = 1000
n_steps = 50000
q_cuda, p_cuda = es.potentials.lagevin_harmonic_lj_cuda(*random_initializer(n_particles, n_dim),
                                                        n_steps, cutoff, damping=damping, time_step=0.001)


In [ ]:
ipv.figure()
s = ipv.scatter(q_cuda[:, :, 0], q_cuda[:, :, 1], q_cuda[:, :, 2], marker='sphere', size=10)
ipv.xyzlim(0, 20)
ipv.animation_control(s) # shows controls for animation controls
ipv.show()

In [ ]:
def _grid(ns):
    xx = np.meshgrid(*[np.arange(0, n) for n in ns])
    X = np.vstack([v.reshape(-1) for v in xx]).T
    return X

def intializer_NaCl(n):
    n_particles = n * n * n
    n_dim = 3
    l_box = 4. * np.array([n, n, n])
    grid=_grid([n, n, n])
    q_0 = 4. * grid
    p_0 = np.zeros((n_particles, n_dim))
    particle_info = grid.sum(axis=1)%2
    mixing_conditions = tuple([(2.35, 2.35**6, 0.130, 1),
                               (0.5 * (2.35 + 4.4), (0.5 * (2.35 + 4.4))**6, 0.13, -1),
                               (0.5 * (2.35 + 4.4), (0.5 * (2.35 + 4.4))**6, -1),
                               (4.40, 4.40**6, 0.1, 1),
                              ])
    masses = [22.990, 35.453] * int(n_particles / 2)
    charges = [1, -1] * int(n_particles / 2)
    return q_0, p_0, np.array(masses), np.array(charges), particle_info, mixing_conditions, np.array(l_box)
cutoff_lj = 4.40 * 3.5

In [ ]:
n_steps = 1000

q_cuda, p_cuda = es.potentials.lagevin_coulomb_lj_cuda(*intializer_NaCl(12), n_steps, cutoff_lj, accuracy=1e-8, damping=0.1, 
                        time_step=0.001, temp=150, k_B=0.00198720360, k_C=332.0637128,)

In [ ]:
# right now its neither fast nor does it look sensible =(

q_cuda_mod = q_cuda % 48
ipv.figure()
s = ipv.scatter(q_cuda_mod[:, :, 0], q_cuda_mod[:, :, 1], q_cuda_mod[:, :, 2], marker='sphere', size=10)
ipv.xyzlim(0, 48)
ipv.animation_control(s) # shows controls for animation controls
ipv.show()